<a href="https://colab.research.google.com/github/S-ArunaDevi06/image_defogging/blob/main/Image_Defogging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#os.chdir('/content/drive/MyDrive/Colab Notebooks/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Function to estimate the dark channel of an image
def dark_channel(img, size=15):
    min_channel = np.min(img, axis=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    dark = cv2.erode(min_channel, kernel)
    return dark

# Function to estimate the atmospheric light
def atmospheric_light(img, dark):
    [h, w] = img.shape[:2]
    num_pixels = h * w
    top_pixels = int(max(np.floor(num_pixels * 0.001), 1))
    dark_vec = dark.reshape(num_pixels)
    img_vec = img.reshape(num_pixels, 3)

    indices = dark_vec.argsort()[::-1][:top_pixels]
    atmospheric_light = np.mean(img_vec[indices], axis=0)
    return atmospheric_light

# Function to estimate the transmission map
def estimate_transmission(img, atmospheric_light, omega=0.90, size=15):
    norm_img = img / atmospheric_light
    transmission = 1 - omega * dark_channel(norm_img, size)
    return transmission

# Function to recover the haze-free image
def recover_image(img, transmission, atmospheric_light, t_min=0.1):
    transmission = np.clip(transmission, t_min, 1)
    J = (img - atmospheric_light) / transmission[:, :, np.newaxis] + atmospheric_light
    J = np.clip(J, 0, 255).astype(np.uint8)
    return J

# Function to sharpen an image using a Laplacian kernel
def sharpen_image(img):
    kernel = np.array([[0, -1, 0],
                       [-1, 5.5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(img, -1, kernel)
    return sharpened

# Function to brighten an image
def brighten_image(img, beta=50):
    brightened = cv2.convertScaleAbs(img, alpha=1, beta=beta)
    return brightened

# Directory paths (replace with your actual directories)
input_folder = '/content/drive/MyDrive/IP_DATASET'  # Input folder containing images
output_folder = '/content/drive/MyDrive/defogged_and_sharpened'  # Output folder to save processed images

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get all image files in the folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Process each image in the folder
for image_file in image_files:
    # Load the image
    img_path = os.path.join(input_folder, image_file)
    img = cv2.imread(img_path)


    if img is None:
        print(f"Failed to load {image_file}")
        continue

    # Step 1: Defog the image using Dark Channel Prior
    dark = dark_channel(img)
    A = atmospheric_light(img, dark)
    transmission = estimate_transmission(img, A)
    recovered_img = recover_image(img, transmission, A)

    # Step 2: Sharpen the image
    sharpened_img = sharpen_image(recovered_img)

    # Step 3: Brighten the image (optional)
    brightened_img = brighten_image(sharpened_img, beta=50)

    #plt.imshow()
    #plt.show()

    # Save the final processed image
    output_img_path = os.path.join(output_folder, image_file)
    cv2.imwrite(output_img_path, brightened_img)

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Input Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(brightened_img)
    plt.title('Output Image')
    plt.axis('off')

    plt.show()
    print(f"Processed and saved: {output_img_path}")

print("Processing complete!")


In [ ]:
# Function to estimate the dark channel of an image

import cv2
import numpy as np
from skimage.metrics import structural_similarity as compare_ssim
from math import log10, sqrt

# Function to calculate PSNR (Peak Signal-to-Noise Ratio)
def calculate_psnr(original_img, processed_img):
    mse = np.mean((original_img - processed_img) ** 2)
    if mse == 0:  # MSE is zero means no noise is present in the signal.
        return float('inf')
    max_pixel = 255.0  # Maximum pixel value for 8-bit images
    psnr_value = 20 * log10(max_pixel / sqrt(mse))
    return psnr_value

# Function to calculate SSIM (Structural Similarity Index)
def calculate_ssim(original_img, processed_img):
    original_img_gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    processed_img_gray = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
    ssim_value, _ = compare_ssim(original_img_gray, processed_img_gray, full=True)
    return ssim_value

def dark_channel(img, size=15):
    min_channel = np.min(img, axis=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    dark = cv2.erode(min_channel, kernel)
    return dark

# Function to estimate the atmospheric light
def atmospheric_light(img, dark):
    [h, w] = img.shape[:2]
    num_pixels = h * w
    top_pixels = int(max(np.floor(num_pixels * 0.001), 1))
    dark_vec = dark.reshape(num_pixels)
    img_vec = img.reshape(num_pixels, 3)

    indices = dark_vec.argsort()[::-1][:top_pixels]
    atmospheric_light = np.mean(img_vec[indices], axis=0)
    return atmospheric_light

# Function to estimate the transmission map
def estimate_transmission(img, atmospheric_light, omega=0.95, size=15):
    norm_img = img / atmospheric_light
    transmission = 1 - omega * dark_channel(norm_img, size)
    return transmission

# Function to recover the haze-free image
def recover_image(img, transmission, atmospheric_light, t_min=0.1):
    transmission = np.clip(transmission, t_min, 1)
    J = (img - atmospheric_light) / transmission[:, :, np.newaxis] + atmospheric_light
    J = np.clip(J, 0, 255).astype(np.uint8)
    return J

# Function to sharpen an image using a Laplacian kernel
def sharpen_image(img):
    kernel = np.array([[0, -1, 0],
                       [-1, 5.5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(img, -1, kernel)
    return sharpened

# Function to brighten an image
def brighten_image(img, beta=50):
    brightened = cv2.convertScaleAbs(img, alpha=1, beta=beta)
    return brightened

# Directory paths (replace with your actual directories)
input_folder = '/content/drive/MyDrive/foggy'  # Input folder containing images
output_folder = '/content/drive/MyDrive/foggy_defogged_and_sharpened'  # Output folder to save processed images

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get all image files in the folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
count=0

# Calculate PSNR and SSIM

psnr, ssim=[],[]


# Process each image in the folder
while (image_files[count] in image_files) and (count<500):
    # Load the image
    img_path = os.path.join(input_folder, image_files[count])
    img = cv2.imread(img_path)


    if img is None:
        print(f"Failed to load {image_files[count]}")
        continue

    # Step 1: Defog the image using Dark Channel Prior
    dark = dark_channel(img)
    A = atmospheric_light(img, dark)
    transmission = estimate_transmission(img, A)
    recovered_img = recover_image(img, transmission, A)

    # Step 2: Sharpen the image
    sharpened_img = sharpen_image(recovered_img)

    # Step 3: Brighten the image (optional)
    brightened_img = brighten_image(sharpened_img, beta=50)

    #plt.imshow()
    #plt.show()

    # Save the final processed image
    output_img_path = os.path.join(output_folder, image_files[count])
    cv2.imwrite(output_img_path, brightened_img)

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(brightened_img)
    plt.title('Brightened Image')
    plt.axis('off')

    plt.show()

    psnr_value = calculate_psnr(img, brightened_img)
    ssim_value = calculate_ssim(img, brightened_img)

    psnr.append(psnr_value)
    ssim.append(ssim_value)

    print(f"Processed and saved: {output_img_path}")

    print(count)

    count+=1

print("Processing complete!")
print("psnr: ",np.mean(psnr))
print("ssim: ",np.mean(ssim))
